In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/Cifar10.zip -d /content/drive/MyDrive/Cifar10

In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers, models, losses, optimizers, datasets, preprocessing, utils

from ipywidgets import interact
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
groups_folder_path = '/content/drive/MyDrive/Cifar10/'
categories = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

num_classes = len(categories)

import cv2
import os

image_w = 32
image_h = 32

X = []
Y = []

for idex, categorie in enumerate(categories):
    label = [0 for i in range(num_classes)]
    label[idex] = 1
    image_dir = groups_folder_path + categorie + '/'

    for top, dir, f in os.walk(image_dir):
        for filename in f:
            #print(image_dir+filename) 
            img = cv2.imread(image_dir+filename) #imread : 파일 하나 읽어주기
            img = cv2.resize(img, None, fx=image_w/img.shape[0], fy=image_h/img.shape[1]) # 여기서 normalizing해줌
            X.append(img/255)
            Y.append(label)
 

X = np.array(X)
Y = np.array(Y)
print(X.shape)
print(Y.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dropout, Activation, Dense
from keras.layers import Flatten, Convolution2D, MaxPooling2D
from keras.models import load_model
from sklearn.model_selection import train_test_split


X_train_data, X_test_data, Y_train_label, Y_test_label = train_test_split(X,Y, stratify=Y) #stratify를 통해서 한 쪽에 쏠려서 분배되는 것을 방지
xy = (X_train_data, X_test_data, Y_train_label, Y_test_label)

np.save("./img_data.npy", xy) # train과 test set으로 나누어진 파일을 numpy 파일에 넣어줌
X_train_data.shape, X_test_data.shape, Y_train_label.shape, Y_test_label.shape

In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers, models, losses, optimizers, datasets, preprocessing, utils
from ipywidgets import interact
import matplotlib.pyplot as plt
import pandas as pd

input_layer = layers.Input((32, 32, 3))

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt


model = keras.models.Sequential( [
    keras.layers.Conv2D(input_shape = (32, 32, 3),
                        kernel_size = (4,4), padding = 'same', 
                        filters = 32),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(kernel_size = (4,4), padding = 'same', 
                        filters = 32),
    keras.layers.BatchNormalization(),
     keras.layers.MaxPooling2D((2, 2)), # maxpooling크기 변경 / stride 변경
     keras.layers.Dropout(rate=0.4),
     keras.layers.Conv2D(kernel_size = (3,3), padding ='same', 
                        filters = 64),
     keras.layers.BatchNormalization(),
     keras.layers.Conv2D(kernel_size = (3,3), padding ='same', 
                        filters = 64),
     keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(rate=0.4),
    keras.layers.Conv2D(kernel_size = (3,3), padding = 'same', 
                        filters = 128),
     keras.layers.BatchNormalization(),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(10, activation = 'softmax'),
])

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss=losses.CategoricalCrossentropy(),
    optimizer='adam',
    metrics=["accuracy"],
)

In [ ]:
history = model.fit(
    X_train_data, Y_train_label,
    validation_data=(X_test_data, Y_test_label),
    batch_size=256,
    epochs=50
)

In [ ]:
pd.DataFrame(history.history).plot()
# batch normalization 2개에 하나씩 사용한 결과

In [ ]:
plt.plot(history.history['loss'], 'b-') # 파란선 : 훈련용 데이터에 대한 손실
plt.plot(history.history['val_loss'], 'r--') # 빨간선 : 검증용 데이터에 대한 손실

In [ ]:
plt.plot(history.history['accuracy'], 'b-') # 파란선 : 훈련용 데이터에 대한 정확도
plt.plot(history.history['val_accuracy'], 'r--') # 빨간선 : 검증용 데이터에 대한 정확도